In [33]:
# -------------------------
# Part 0 - Remove corrupted images
# -------------------------
from PIL import Image, UnidentifiedImageError
import os
# Image: Used to open and verify image files.
# UnidentifiedImageError: Raised when PIL(Pillow) fails to recognize a file as an image.
# os: Used to navigate the file system.

def remove_corrupted_images(folder_path):   #Defines a function called remove_corrupted_images that takes a folder path as input.
    deleted = 0
    for root, _, files in os.walk(folder_path):                            # os.walk() recursively walks through the folder and all its subfolders.root: current directory path._: list of subdirectories (not used here, so it’s _).files: list of files in the current root.
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):         #Checks if the file is an image based on its extension (case-insensitive).Only processes supported image types.
                file_path = os.path.join(root, file)         #Joins the folder path and file name to get the full path to the image.
                try:                            #Tries to:Open the image using Image.open().Call .verify() to ensure the image is not corrupted (doesn’t load pixels, just checks header and structure).
                    img = Image.open(file_path)             
                    img.verify()  # Verify image file integrity
                except (IOError, UnidentifiedImageError, SyntaxError) as e:           #If the image is unreadable or corrupted, it may raise:IOError: generic file read error.UnidentifiedImageError: image file can’t be opened as an image.SyntaxError: malformed image (older PIL versions sometimes throw this).
                    print(f"❌ Deleting corrupted file: {file_path} — {e}")            #Prints which file is being deleted and why.
                    os.remove(file_path)         #Deletes the corrupted image file.
                    deleted += 1     #Increments the deleted counter.
    print(f"\n✅ Done. Total corrupted images removed from {folder_path}: {deleted}")        #After the loop, prints a summary of how many images were removed from the folder.

In [34]:
# Clean corrupted images before training
remove_corrupted_images("cnnData/training_set")
remove_corrupted_images("cnnData/test_set")


✅ Done. Total corrupted images removed from cnnData/training_set: 0

✅ Done. Total corrupted images removed from cnnData/test_set: 0


In [35]:
# -------------------------
# Part 1 - CNN Setup & Preprocessing
# -------------------------
import tensorflow as tf         #TensorFlow is a deep learning framework used for building and training models like CNNs.
from tensorflow.keras.preprocessing.image import ImageDataGenerator         #Imports ImageDataGenerator, which is a powerful tool to:Load images from folders,Augment images (rotate, flip, zoom, etc.),Normalize pixel values (like rescaling to 0–1).
import numpy as np                  #It’s used later to manipulate image arrays (e.g. np.expand_dims to convert an image into a batch of size 1).
from keras.preprocessing import image              #Imports the image module from Keras to help- Load and preprocess single images,Convert images to arrays for prediction using image.load_img() and image.img_to_array().
print("✅ TensorFlow version:", tf.__version__)

✅ TensorFlow version: 2.19.0


In [36]:
# Preprocess Training Set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# rescale=1./255: Normalizes image pixel values from 0–255 to 0–1, which helps neural networks converge faster.
# shear_range=0.2: Randomly shears (distorts) the image along an axis, making the model more robust to skewed perspectives.
# zoom_range=0.2: Randomly zooms into images, helping the model learn better scale invariance.
# horizontal_flip=True: Randomly flips images left-to-right — useful for symmetrical objects (e.g., cats/dogs facing either way).
                                                                  
training_set = train_datagen.flow_from_directory('cnnData/training_set',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

# flow_from_directory(...): Loads images from folders on disk.
# 'cnnData/training_set':Folder should contain subfolders like /cats, /dogs.Each subfolder name becomes a class label.
# target_size=(64, 64):Resizes every image to 64×64 pixels before feeding to the model.Your CNN is built to accept input shape (64, 64, 3).
# batch_size=32:Loads and processes 32 images at a time (faster & memory-efficient).
# class_mode='binary':Used for 2 classes only (e.g., cat vs dog).Labels are 0 and 1.

Found 6501 images belonging to 2 classes.


In [37]:
# Preprocess Test Set
test_datagen = ImageDataGenerator(rescale=1./255)

# Creates a test_datagen object to handle test image preprocessing.
# rescale=1./255:Normalizes image pixel values from [0–255] to [0–1].
# This is essential to match the training image format. No augmentations (like flip/zoom) are applied to test data — test data must reflect real-world, untouched inputs.

test_set = test_datagen.flow_from_directory('cnnData/test_set',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

# Loads and preprocesses test images from 'cnnData/test_set'.
# target_size=(64, 64):Resizes all test images to 64×64 pixels to match the CNN input shape.
# batch_size=32:Loads 32 images at a time during validation.
# class_mode='binary':Labels are binary: 0 or 1, suitable for 2-class classification (e.g., cats vs dogs)

Found 143 images belonging to 2 classes.


In [38]:
# -------------------------
# Part 2 - Build the CNN
# -------------------------
cnn = tf.keras.models.Sequential([              #Initializes a Sequential model, meaning layers will be stacked one after another.
    tf.keras.Input(shape=(64, 64, 3)),           #Defines the input shape:64 x 64 pixels (as per preprocessing).3 channels → for RGB images.This is the shape of each image entering the model.
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),           #Applies 32 filters of size 3×3 to the input image.Captures features like edges, textures, etc.activation='relu': Applies the ReLU function, introducing non-linearity.
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),                #Max Pooling layer:Reduces image size by taking the max value in each 2×2 window.Helps reduce computation and overfitting by downsampling.
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),                
    #Another Conv + MaxPooling block:Deepens the model and allows it to learn more complex patterns.Still uses 32 filters of size 3×3 and same downsampling logic.
    tf.keras.layers.Flatten(),          #Converts the 2D feature maps from the last layer into a 1D vector.This is needed to pass it to the fully connected (dense) layers.
    tf.keras.layers.Dense(units=128, activation='relu'),    
    #Fully connected hidden layer:128 neurons (a common starting point).Uses ReLU activation to learn complex patterns from extracted features.
    tf.keras.layers.Dense(units=1, activation='sigmoid')  #Output layer for binary classification:1 neuron (since only 2 classes: e.g., cat vs dog).Sigmoid activation gives output between 0 and 1 → interpreted as class probability.
])

In [39]:
# -------------------------
# Part 3 - Compile & Train
# -------------------------
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# This line prepares the model for training by setting:
#optimizer='adam'
# Uses the Adam optimizer (Adaptive Moment Estimation).
# Combines benefits of SGD + RMSProp.
# Automatically adjusts the learning rate → works well in most deep learning tasks.
# 🔹 loss='binary_crossentropy'
# Suitable for binary classification (two output classes: e.g., cats vs dogs).
# Measures how far the predicted probability is from the true label (0 or 1).
# 🔹 metrics=['accuracy']
# Tracks accuracy during training and validation.
# Accuracy = % of correct predictions.

cnn.fit(x=training_set, validation_data=test_set, epochs=25)
#This line starts the training of your CNN model.
# 🔹 x=training_set
# The training data generator you created earlier.
# Loads and feeds batches of images and labels to the model.
# 🔹 validation_data=test_set
# Validation data generator.
# Model evaluates its performance on this set after each epoch (to monitor generalization).
# 🔹 epochs=25
# The number of times the model will see the entire training dataset.
# More epochs can improve accuracy but risk overfitting.

Epoch 1/25
204/204 ━━━━━━━━━━━━━━━━━━━━ 63s 298ms/step - accuracy: 0.5877 - loss: 0.6696 - val_accuracy: 0.5594 - val_loss: 0.6788
Epoch 2/25
204/204 ━━━━━━━━━━━━━━━━━━━━ 68s 335ms/step - accuracy: 0.6656 - loss: 0.6049 - val_accuracy: 0.6503 - val_loss: 0.6051
Epoch 3/25
204/204 ━━━━━━━━━━━━━━━━━━━━ 72s 353ms/step - accuracy: 0.7006 - loss: 0.5597 - val_accuracy: 0.7063 - val_loss: 0.5290
Epoch 4/25
204/204 ━━━━━━━━━━━━━━━━━━━━ 67s 328ms/step - accuracy: 0.7198 - loss: 0.5481 - val_accuracy: 0.7483 - val_loss: 0.5247
Epoch 5/25
204/204 ━━━━━━━━━━━━━━━━━━━━ 59s 290ms/step - accuracy: 0.7308 - loss: 0.5238 - val_accuracy: 0.7483 - val_loss: 0.4508
Epoch 6/25
204/204 ━━━━━━━━━━━━━━━━━━━━ 55s 270ms/step - accuracy: 0.7570 - loss: 0.4941 - val_accuracy: 0.7552 - val_loss: 0.4928
Epoch 7/25
204/204 ━━━━━━━━━━━━━━━━━━━━ 67s 327ms/step - accuracy: 0.7775 - loss: 0.4649 - val_accuracy: 0.7902 - val_loss: 0.4328
Epoch 8/25
204/204 ━━━━━━━━━━━━━━━━━━━━ 56s 272ms/step - accuracy: 0.7828 - loss: 0

In [40]:
# accuracy - How well the model is doing on the training data. E.g. 0.9023 means 90.23% correct on the training set.
# loss - A number that shows how bad the model's predictions are on training data. Lower is better.
# val_accuracy - Accuracy on the test (validation) data — how well it generalizes to unseen data.
# val_loss - Loss on the test/validation data — again, lower is better.

# Training and validation accuracy are both above 90%.
# Validation loss is low and stable, meaning it's not overfitting.
# The gap between accuracy and val_accuracy is small — your model generalizes well.
# You're not seeing signs of overfitting (e.g., high training acc but low val acc).

In [43]:
# -------------------------
# Part 4 - Single Prediction
# -------------------------
img_path = os.path.join('cnnData', 'prediction', 'cat.69.jpg')     #Builds the path to your image: cnnData/prediction/cat.69.jpg.Using os.path.join makes it platform-safe (Windows/Linux).

try:
    test_image = image.load_img(img_path, target_size=(64, 64))          #Loads the image and resizes it to the expected model input size (64x64 pixels).
    test_image = image.img_to_array(test_image)           #Converts the image to a NumPy array (height × width × channels).
    test_image = np.expand_dims(test_image, axis=0)          
    #Adds an extra dimension → converts shape from (64, 64, 3) to (1, 64, 64, 3).This mimics a batch of size 1, required by model.predict().
    #When you trained your CNN, you gave it batches of images shaped like:(batch_size, height, width, channels)→ e.g. (32, 64, 64, 3)
    #Even if you want to predict just one image, the model still expects a batch, not a single image.  So you convert:(64, 64, 3) → (1, 64, 64, 3)

    result = cnn.predict(test_image)   #Predicts the class probability using the CNN.

    print("Raw output:", result)  #Shows the raw model output (e.g. [[0.86]]).

    if result[0][0] >= 0.5:
        prediction = 'dog'
    else:
        prediction = 'cat'

    #If the output is ≥ 0.5, it's considered class 1 (dog).Otherwise, class 0 (cat).TensorFlow assigns labels based on directory names in alphabetical order:So by default:cat = 0, dog = 1

    print("Predicted class:", prediction)

except (UnidentifiedImageError, IOError) as e:
    print(f"❌ Could not read prediction image: {img_path} — {e}")      #If the image is corrupted or unreadable, it shows a helpful error message instead of crashing the code.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Raw output: [[1.]]
Predicted class: dog
